In [1]:
import os, sys
# Root directory of the project
ROOT_DIR = os.path.abspath("../")

# To find local version of the library
sys.path.append(ROOT_DIR)

import dateutil.tz
import datetime
import pprint

import skopt
from skopt import gp_minimize


import numpy as np
import torch
from torchvision import transforms


from utils.dataloader import SVHNDataset
from utils.config import cfg, cfg_from_file
from utils.dataloader import prepare_dataloaders
from utils.misc import mkdir_p
from utils.transforms import FirstCrop, Rescale, RandomCrop, ToTensor
from utils.misc import load_obj
from utils.visualization import visualize_sample

from models.vgg import VGG
from trainer.trainer import train_model

from utils.checkpointer import CheckpointSaver

import matplotlib.pyplot as plt
%matplotlib inline

%load_ext autoreload
%autoreload 2


In [6]:
cfg_from_file("../config/base_config.yml")

vgg19 = VGG('VGG19', num_classes_length=7, num_classes_digits=10)

checkpoint = CheckpointSaver("../tmp_results")
checkpoint.save(vgg19, 11)

model = checkpoint.load("checkpoint_epoch11")

print(model)
print(cfg)

Init VGG
Checkpointing new model ...
VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14)

In [4]:
# Wrapper function for bayesian optimization
# The space contains the parameters to explore
def train_model_opt(space):
    print(space)
    (train_loader,
     valid_loader) = prepare_dataloaders(
        dataset_split='train',
        dataset_path='../data/SVHN/train',
        metadata_filename='../data/SVHN/train_metadata.pkl',
        batch_size=32,
        sample_size=1000,
        valid_split=0.8)
    
    vgg19 = VGG('VGG19', num_classes_length=7, num_classes_digits=10)


    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("Device used: ", device)
    return -train_model(vgg19,
                train_loader=train_loader,
                valid_loader=valid_loader,
                num_epochs=2,
                device=device,
                lr=space[0],
                checkpoint_dir="../checkpoints",
                output_dir="../tmp_results")


# Define the range 
space = [skopt.space.Real(10**-5, 10**0, "log-uniform", name='lr'), 
         skopt.space.Categorical(["VGG11", "VGG13", "VGG16", "VGG19"])]

mkdir_p("../tmp_results")


res_gp = gp_minimize(train_model_opt, space, n_calls=10,
                     random_state=0)

print("Best score: {0}".format(-res_gp.fun))
print("Best lr: {0}".format(res_gp.x[0]))

[0.009209225155490905, 'VGG19']
Init VGG
Device used:  cuda:0
Learning rate is: 0.009209225155490905
Directory run already exists
# Start training #


Iterating over training data...


  0%|          | 0/7 [00:00<?, ?it/s]



Iterating over validation data...


100%|██████████| 7/7 [00:04<00:00,  1.28s/it]


Checkpointing new model ...

Epoch: 1/2
	Train Loss: 10.9502
	Valid Loss: 437.2933
	Valid Accuracy: 0.0250


Iterating over training data...


...

Checkpointing new model ...

Epoch: 1/2
	Train Loss: 10.5590
	Valid Loss: 10.1405
	Valid Accuracy: 0.0000


Iterating over training data...


  0%|          | 0/7 [00:00<?, ?it/s]



Iterating over validation data...


100%|██████████| 7/7 [00:01<00:00,  3.67it/s]



Epoch: 2/2
	Train Loss: 9.4754
	Valid Loss: 9.0634
	Valid Accuracy: 0.0300


Training complete in 0m 20s
Saving model ...
Best model saved to : ../tmp_results/best_model.pth
Best score: 0.035
Best lr: 0.0006937575128144787
